In [18]:
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [5]:
def draftkings_parse(input_dict, driver, key):
    outcomes = [x.text for x in driver.find_elements_by_class_name(input_dict['outcomes_class'])]
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    odds = [x.text for x in soup.findAll('span',{'class', input_dict['odds_class']})]
    # odds = [x.text for x in driver.find_elements_by_class_name(input_dict['odds_class'])]
    return list(zip(outcomes[::3], outcomes[2::3], odds[::3], odds[1::3], odds[2::3], [key] * len(outcomes[::3]), [datetime.now()] * len(outcomes[::3])))

In [6]:
prem_dir = {'draftkings':{'url':'https://sportsbook.draftkings.com/leagues/soccer/53591936',
                          'parser': draftkings_parse,
                          'args':{'outcomes_class':'sportsbook-outcome-cell__label',
                                  'odds_class':'sportsbook-odds american default-color'}}
           }
           
output_cols = ['home_team','away_team', 'home_odds','tie_odds', 'away_odds', 'source', 'datetime']

In [14]:
options = Options()
options.add_argument('--headless')
options.add_argument('--window-size=1920,1200')
DRIVER_PATH = 'driver/mac/chromedriver'

In [19]:
dn = []
for key in prem_dir:
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
    driver.get(prem_dir[key]['url'])
    driver.implicitly_wait(3)
    
    data = pd.DataFrame(prem_dir[key]['parser'](prem_dir[key]['args'],driver, key),columns=output_cols)
    dn.append(data)
    driver.quit()
df = pd.concat(dn, axis=0)

In [33]:
df.head().to_dict(orient='records')[0]

{'home_team': 'Brighton',
 'away_team': 'West Bromwich',
 'home_odds': '-143',
 'tie_odds': '+300',
 'away_odds': '+370',
 'source': 'draftkings',
 'datetime': Timestamp('2020-10-26 00:23:58.195356')}

In [25]:
from flask_restful import Resource, Api
